# 2: Two-Step Method using FinBERT as Machine Learning Model

In this script, the two-step strategy is applied to the dataset sp500_headlines_2008_2024.csv:
- Each article headline is evaluated with the FinBERT model, yielding a sentiment score for the headline.
- For each trading day, the average sentiment across all headlines is computed, where the number of available headlines on a given day serves as the measure C_i.
- The daily sentiment estimate is then used as a generated regressor in a downstream economic model via simple linear regression.
- To assess the role of C_i, the sample is split into three groups (low, medium, high number of headlines) and the regression is estimated separately for each group.


### Import required packages

In [1]:
# Two-Step: Headlines -> Sentiment (θ̂t) -> Returns regression
# Erwartete Spalten in deiner CSV: 'Date', 'Headline', 'CP'
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from tqdm import tqdm
import pandas as pd
import statsmodels.api as sm

c:\Users\49176\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Prepare Data

In [ ]:
df = pd.read_csv('../data/sp500_headlines_2008_2024.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)

### Load FinBERT

In [3]:
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### Compute Sentiments

In [4]:
# Define batch size
batch_size = 32
sentiments = []

# Plug Headlines into FinBERT
for i in tqdm(range(0, len(df), batch_size)):
    batch = df["Headline"].iloc[i:i+batch_size].astype(str).tolist()
    inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=128)

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # FinBERT labels: 0 = negative, 1 = neutral, 2 = positive
    batch_scores = probs[:, 2] - probs[:, 0]  # Positive - Negative = Sentiment Score
    sentiments.extend(batch_scores.numpy())

df["finbert_sent"] = sentiments

100%|██████████| 598/598 [20:19<00:00,  2.04s/it]


### Aggregation

In [5]:
# Aggregation: take mean of the raw sentiment estimate for every day (average of duplicate entries)
# Note: choosing median for a more robust result is an other option here
daily = (
    df.groupby('Date', as_index=False)
      .agg(hat_theta=('finbert_sent','mean'),
           CP=('CP','last'),
           Return_simple=('Return','last'),
           Return_log=('Return_log','last'),
           n_headlines=('Headline','size'))
)

### Downstream economic model

In [6]:
# OLS with Newey-West standard errors (HAC=heteroskedasticity and autocorrelation consistent)
# Basismodell as introduced by Tetlock: R_t = γ0 + γ1 * θ̂_t + ε_t
def ols_hac(y, X, lags=5):
    Xc = sm.add_constant(X)
    return sm.OLS(y, Xc, missing='drop').fit(cov_type='HAC', cov_kwds={'maxlags': lags})

results_by_C = {}

# Define groups for the amount of unstructered data (n_headlines): low, medium, high
daily['C_group'] = pd.qcut(daily['n_headlines'], q=3, labels=["low", "medium", "high"])

for grp, dailyg in daily.groupby("C_group"):
    X = sm.add_constant(dailyg['hat_theta'])
    y = dailyg['Return_simple']
    model = sm.OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': 5})
    results_by_C[grp] = model

# Print results
for grp, model in results_by_C.items():
    print(f"=== Group: {grp} ===")
    print(model.summary().tables[1])
    print("\n")


C:\Users\49176\AppData\Local\Temp\ipykernel_12036\2476678587.py:12: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for grp, dailyg in daily.groupby("C_group"):


=== Group: low ===
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.663e-05      0.000     -0.061      0.952      -0.001       0.001
hat_theta     -0.0002      0.001     -0.369      0.712      -0.001       0.001


=== Group: medium ===
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const      -6.616e-06      0.000     -0.033      0.973      -0.000       0.000
hat_theta     -0.0002      0.000     -0.683      0.495      -0.001       0.000


=== Group: high ===
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0004      0.000      2.177      0.029    3.55e-05       0.001
hat_theta      0.0004      0.000      1.179      0.238      -0.000

In [8]:
df

,Headline,Date,CP,Return,Return_log,finbert_sent
0,"JPMorgan Predicts 2008 Will Be ""Nothing But Net""",2008-01-02,1447.16,NaN,NaN,-9.849703e-01
1,Dow Tallies Biggest First-session-of-year Poin...,2008-01-02,1447.16,0.000000,0.000000,4.291815e-01
2,2008 predictions for the S&P 500,2008-01-02,1447.16,0.000000,0.000000,-9.992520e-01
3,"U.S. Stocks Higher After Economic Data, Monsan...",2008-01-03,1447.16,0.000000,0.000000,-1.243351e-06
4,U.S. Stocks Climb As Hopes Increase For More F...,2008-01-07,1416.18,-0.021407,-0.021640,-1.610295e-02
...,...,...,...,...,...,...
19122,"Dow Jones, S&P 500 end lower as investors awai...",2024-03-04,5130.95,0.000000,0.000000,7.008778e-01
19123,S&P 500 edges lower as investors hold their br...,2024-03-04,5130.95,-0.001193,-0.001194,3.410302e-01
19124,S&P 500 Price Forecast – S&P 500 Continues to ...,2024-03-04,5130.95,0.000000,0.000000,-3.340593e-08
19125,"Super Micro, Deckers to join S&P 500 index",2024-03-04,5130.95,0.000000,0.000000,-9.998519e-01


### Export final dataset as csv

In [9]:
daily.to_csv("../data/daily_headlines_embedded_FinBERT.csv", index=False, encoding="utf-8")
df.to_csv("../data/sp500_headlines_2008_2024_FinBERT.csv", index=False, encoding="utf-8")